In [96]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [97]:
def conv_block(inputs,filters,kernel_size,strides):
  conv_1 = layers.Conv2D(filters=filters,kernel_size=kernel_size,strides=strides,padding='same')(inputs)
  bn_1 = layers.BatchNormalization()(conv_1)
  relu_1 = layers.ReLU()(bn_1)
  return relu_1

In [98]:
def deptwise_seperable_conv_block(inputs,filters,kernel_size,strides):
  #depth wise conv
  dept_conv_1 =layers.DepthwiseConv2D(kernel_size=kernel_size,strides=strides,padding='same')(inputs)
  bn_2 = layers.BatchNormalization()(dept_conv_1)
  relu_2 = layers.ReLU()(bn_2)
  # pointwise conv
  point_conv = layers.Conv2D(filters=filters,kernel_size=(1,1),strides=1,padding='same')(relu_2)
  bn_3 = layers.BatchNormalization()(point_conv)
  relu_3 = layers.ReLU()(bn_3)
  return relu_3

In [99]:
def MobileNetV1(input_shape, num_classes, activation):
    input_layer = tf.keras.Input(shape=input_shape)

    layer_1 = conv_block(inputs=input_layer, filters=32,kernel_size=(3,3),strides=2)
    layer_2 = deptwise_seperable_conv_block(inputs=layer_1, filters=64, kernel_size=(3,3),strides=1)
    layer_3 = deptwise_seperable_conv_block(inputs=layer_2, filters=128, kernel_size=(3,3),strides=2)
    layer_4 = deptwise_seperable_conv_block(inputs=layer_3, filters=128, kernel_size=(3,3),strides=1)
    layer_5 = deptwise_seperable_conv_block(inputs=layer_4, filters=256, kernel_size=(3,3),strides=2)
    layer_6 = deptwise_seperable_conv_block(inputs=layer_5, filters=256,kernel_size= (3,3),strides= 1)
    layer_7 = deptwise_seperable_conv_block(inputs=layer_6, filters=512, kernel_size=(3,3),strides=2)

    for _ in range(5):
        layer_7 = deptwise_seperable_conv_block(inputs=layer_7, filters=512,kernel_size=( 3,3),strides= 1)

    layer_8 = deptwise_seperable_conv_block(inputs=layer_7,filters= 1024,kernel_size= (3,3),strides=2)
    layer_9 = deptwise_seperable_conv_block(inputs=layer_8, filters=1024, kernel_size=(3,3), strides=1)

    layer_10 = layers.GlobalAveragePooling2D()(layer_9)
    out_put_layer = layers.Dense(units=num_classes, activation=activation)(layer_10)

    model = models.Model(inputs=input_layer, outputs=out_put_layer)

    return model

# Example usage with 3-channel RGB images and 1000 classes (Imagenet):
input_shape = (224, 224, 3)
num_classes = 20
activation = 'softmax'

mobilenetV1 = MobileNetV1(input_shape, num_classes,activation)
mobilenetV1.summary()


Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_38 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_312 (Conv2D)         (None, 112, 112, 32)      896       
                                                                 
 batch_normalization_592 (Ba  (None, 112, 112, 32)     128       
 tchNormalization)                                               
                                                                 
 re_lu_584 (ReLU)            (None, 112, 112, 32)      0         
                                                                 
 depthwise_conv2d_288 (Depth  (None, 112, 112, 32)     320       
 wiseConv2D)                                                     
                                                                 
 batch_normalization_593 (Ba  (None, 112, 112, 32)     128